In [ ]:
!pip install --upgrade datasets
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
from datasets import load_dataset

dataset = load_dataset("ErfanMoosaviMonazzah/fake-news-detection-dataset-English")

In [ ]:
dataset.set_format(type='pandas')
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'],
        num_rows: 8267
    })
})

In [ ]:
dataset['train'][:]

,Unnamed: 0,title,text,subject,date,label
0,2619,Ex-CIA head says Trump remarks on Russia inter...,Former CIA director John Brennan on Friday cri...,politicsNews,"July 22, 2017",1
1,16043,YOU WON’T BELIEVE HIS PUNISHMENT! HISPANIC STO...,How did this man come to OWN this store? There...,Government News,"Jun 19, 2017",0
2,876,Federal Reserve governor Powell's policy views...,President Donald Trump on Thursday tapped Fede...,politicsNews,"November 2, 2017",1
3,19963,SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...,Hillary Clinton ally David Brock is offering t...,left-news,"Sep 17, 2016",0
4,10783,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,Pleading ignorance is a perfect ploy for Nancy...,politics,"May 26, 2017",0
...,...,...,...,...,...,...
29995,6880,U.S. aerospace industry urges Trump to help Ex...,The chief executive of the U.S. Aerospace Indu...,politicsNews,"December 6, 2016",1
29996,17818,Highlights: Hong Kong leader Carrie Lam delive...,The following are highlights of the maiden pol...,worldnews,"October 11, 2017",1
29997,5689,Obama Literally LAUGHS At Claims That Brexit M...,If there s one thing President Barack Obama is...,News,"June 28, 2016",0
29998,15805,Syrian army takes full control of Deir al-Zor ...,The Syrian army and its allies have taken full...,worldnews,"November 2, 2017",1


In [ ]:
dataset = dataset.remove_columns(["Unnamed: 0", "date", "subject"])
dataset['train'][:]

,title,text,label
0,Ex-CIA head says Trump remarks on Russia inter...,Former CIA director John Brennan on Friday cri...,1
1,YOU WON’T BELIEVE HIS PUNISHMENT! HISPANIC STO...,How did this man come to OWN this store? There...,0
2,Federal Reserve governor Powell's policy views...,President Donald Trump on Thursday tapped Fede...,1
3,SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...,Hillary Clinton ally David Brock is offering t...,0
4,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,Pleading ignorance is a perfect ploy for Nancy...,0
...,...,...,...
29995,U.S. aerospace industry urges Trump to help Ex...,The chief executive of the U.S. Aerospace Indu...,1
29996,Highlights: Hong Kong leader Carrie Lam delive...,The following are highlights of the maiden pol...,1
29997,Obama Literally LAUGHS At Claims That Brexit M...,If there s one thing President Barack Obama is...,0
29998,Syrian army takes full control of Deir al-Zor ...,The Syrian army and its allies have taken full...,1


In [ ]:
formatted_dataset = dataset.map(lambda example: {"text": (example["title"] + example["text"])[0], "label": example["label"][0]})
formatted_dataset['train'][:]

Map: 100%|██████████| 8267/8267 [00:15<00:00, 532.35 examples/s]


,text,label
0,Ex-CIA head says Trump remarks on Russia inter...,1
1,YOU WON’T BELIEVE HIS PUNISHMENT! HISPANIC STO...,0
2,Federal Reserve governor Powell's policy views...,1
3,SCOUNDREL HILLARY SUPPORTER STARTS “TrumpLeaks...,0
4,NANCY PELOSI ARROGANTLY DISMISSES Questions on...,0
...,...,...
29995,U.S. aerospace industry urges Trump to help Ex...,1
29996,Highlights: Hong Kong leader Carrie Lam delive...,1
29997,Obama Literally LAUGHS At Claims That Brexit M...,0
29998,Syrian army takes full control of Deir al-Zor ...,1


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

final_dataset = formatted_dataset.map(lambda example: {"text": (model.encode(example["text"]))[0], "label": example["label"][0]})
final_dataset['train'][:]

Map: 100%|██████████| 8267/8267 [40:25<00:00,  3.41 examples/s]


,text,label
0,"[-0.10176071, -0.35597068, -0.29525787, -0.111...",1
1,"[-0.10809571, 0.065860674, -0.38816905, 0.0729...",0
2,"[0.09494142, -0.4001917, 0.06682623, 0.2443553...",1
3,"[-0.15997653, -0.49695134, 0.12417561, -0.0594...",0
4,"[-0.3352175, -0.06324625, -0.14606252, -0.0018...",0
...,...,...
29995,"[0.13882601, -0.25633433, -0.28517345, -0.2398...",1
29996,"[-0.108773194, -0.0017903813, 0.25023493, -0.2...",1
29997,"[0.17818034, -0.22581288, 0.10465889, -0.33093...",0
29998,"[-0.010728372, 0.44606093, -0.10517107, -0.468...",1


In [ ]:
final_dataset.save_to_disk("final_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 8267/8267 [00:01<00:00, 7373.06 examples/s] 


In [ ]:
from datasets import load_from_disk

final_dataset = load_from_disk("final_dataset")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

input_data = np.stack(final_dataset["train"]['text'].to_numpy())
output_data = final_dataset["train"]['label'].to_numpy()

# Convert NumPy arrays to TensorFlow tensors
input_tensor = tf.constant(input_data, dtype=tf.float32)
output_tensor = tf.constant(output_data, dtype=tf.float32)

# Define your model architecture
model = models.Sequential([
    layers.Dense(128, input_shape=(384,), activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(input_tensor, output_tensor, epochs=10, batch_size=32, validation_split=0.5)


Epoch 1/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1918 - accuracy: 0.9243 - val_loss: 0.1334 - val_accuracy: 0.9482
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1049 - accuracy: 0.9607 - val_loss: 0.1071 - val_accuracy: 0.9599
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0735 - accuracy: 0.9745 - val_loss: 0.0984 - val_accuracy: 0.9639
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0527 - accuracy: 0.9811 - val_loss: 0.1037 - val_accuracy: 0.9638
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0356 - accuracy: 0.9879 - val_loss: 0.1164 - val_accuracy: 0.9629
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0240 - accuracy: 0.9907 - val_loss: 0.1187 - val_accuracy: 0.9655
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0203 - accuracy: 0.9928 - val_loss: 0.1353 - val_accuracy: 0.9629
Epoch 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7a1aee3d-a63f-4a3b-bb0d-abcb795e3f34' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>